## Reading and getting the df ready etc

In [88]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [89]:
df = pd.read_csv("dataset1.csv")
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


## Helper functions

In [90]:
import pandas as pd
import random


# 1. Train-Test-Split, 
# test size is the percentage of the dataset that should be in the test set
def train_test_split(df, test_size):
    # make sure it is a float and get the number of instances in the test set
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))
    # get the indices for the test set
    indices = df.index.tolist()
    # choose them randomly
    test_indices = random.sample(population=indices, k=test_size)
    # separate into test and train
    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df


# 2. Distinguish categorical and continuous features
def determine_type_of_feature(df):
    # in order to properly split the nodes, we need to know if a feature is categorical or continuous
    feature_types = []
    # this threshold is used to determine if a feature is categorical, it can be changed
    n_unique_values_treshold = 15
    for feature in df.columns:
        # get all features except for the label
        if feature != "label":
            unique_values = df[feature].unique()
            example_value = unique_values[0]
            # we use the number of unique values and the type of the first value to determine the type of the feature
            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types


# 3. Accuracy
def calculate_accuracy(predictions, labels):
    # calculate the accuracy of the predictions
    # we compare the predictions to the labels and count the number of correct predictions
    predictions_correct = predictions == labels
    accuracy = predictions_correct.mean()
    
    return accuracy

# another method to calculate accuracy
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

def precision(y_true, y_pred):
    tp = np.sum(y_true * y_pred)
    fp = np.sum((1 - y_true) * y_pred)
    return tp / (tp + fp)

def sensitivity(y_true, y_pred):
    tp = np.sum(y_true * y_pred)
    fn = np.sum(y_true * (1 - y_pred))
    return tp / (tp + fn)

def specificity(y_true, y_pred):
    tn = np.sum((1 - y_true) * (1 - y_pred))
    fp = np.sum((1 - y_true) * y_pred)
    return tn / (tn + fp)

def f_score(y_true, y_pred):
    tp = np.sum(y_true * y_pred)
    fn = np.sum(y_true * (1 - y_pred))
    fp = np.sum((1 - y_true) * y_pred)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    return 2 * precision * recall / (precision + recall)

# confusion matrix containing true positives, false negatives, false positives and true negatives
def confusion_matrix(y_true, y_pred):
    tp = np.sum(y_true * y_pred)
    fn = np.sum(y_true * (1 - y_pred))
    fp = np.sum((1 - y_true) * y_pred)
    tn = np.sum((1 - y_true) * (1 - y_pred))
    return np.array([[tp, fn], [fp, tn]])


## Decision tree

In [91]:
# 1. Decision Tree helper functions 

# 1.1 Data pure
def check_purity(data):
    # check if labels are pure, i.e. all belong to the same class
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

    
# 1.2 Classify
def classify_data(data):
    # get the labels of the data and return the most common one (majority vote)
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification


# 1.3 Potential splits?
def get_potential_splits(data, random_subspace):
    
    potential_splits = {}
    _, n_columns = data.shape
    column_indices = list(range(n_columns - 1))    # excluding the last column which is the label
    
    # if we decide to take a limited amount of features, we randomly choose a subset of them, this is only used for random forests
    if random_subspace and random_subspace <= len(column_indices):
        column_indices = random.sample(population=column_indices, k=random_subspace)
    
    # we iterate over all columns and find the unique values in each column, in order to determine the splits in our decision tree
    for column_index in column_indices:          
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        potential_splits[column_index] = unique_values
    
    return potential_splits


# 1.4 Lowest Overall Entropy?
def calculate_entropy(data):
    # simply calculate the entropy of the data
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

# this is done to calculate the entropy of the split leaves
def calculate_overall_entropy(data_below, data_above):
    # data below is the left leaf and data above is the right leaf
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n
    
    #calculate both their entropy, sum them and return the overall entropy
    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy


def determine_best_split(data, potential_splits):
    # we iterate over all potential splits and calculate the overall entropy of the split leaves
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            # remove empty lines
            if(len(data_below)==0 or len(data_above)==0):
                break
            
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)
            #compare last calculated overall entropy with the current one, if it is lower, we update the best split
            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    #once we have iterated over all potential splits and compared them, we return the best split
    return best_split_column, best_split_value


# 1.5 Split data
def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]
    #check the type of the feature whether it is continuous or categorical
    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "continuous":
        #if continuous we compare in values larger or smaller
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values >  split_value]
    
    # feature is categorical   
    else:
        #if feature is categorical we compare in values equal or not equal
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values != split_value]
    # we return the leaves
    return data_below, data_above


# 2. Decision Tree Algorithm
def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5, random_subspace=None):
    
    # data preparations
    if counter == 0:
        # we need to store the data globally, so we can use it in the helper functions
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
    
    
    # base cases
    # since our loop is recursive, this will be our end case for a leaf node
    # if the node only has data of one class, we return the class and stop splitting, it will then be a terminal leaf node
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data, random_subspace)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # check for empty data
        if len(data_below) == 0 or len(data_above) == 0:
            classification = classify_data(data)
            # the node cannod be split anymore, so we return the class, it is a terminal node
            return classification
        
        # determine question
        feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        # feature is continuous, we use <=
        if type_of_feature == "continuous":
            question = "{}__<=__{}".format(feature_name, split_value)
            
        # feature is categorical, we use =
        else:
            question = "{}__=__{}".format(feature_name, split_value)
        
        # instantiate sub-tree
        sub_tree = {question: []}
        
        # find answers (recursion)
        # yes is the left leaf, no is the right leaf
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth, random_subspace)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth, random_subspace)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        # we construct the tree and return it
        return sub_tree


# 3. Make predictions
# 3.1 One example
def predict_example(example, tree):
    # this is to predict only one instance
    # get the node questions in order to classify
    question = list(tree.keys())[0]
    #print(question + '\n')
    
    #print(example)
    feature_name, comparison_operator, value = question.split("__")

    # ask question
    if comparison_operator == "<=":
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            # we go to the left leaf
            answer = tree[question][0]
        else:
            # we go to the right leaf
            answer = tree[question][1]

    # base case
    # if the answer is not a sub-tree, that means it's a terminal node, we return the classification
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    # we go deeper into the tree using the sub-tree returned in the answer
    else:
        residual_tree = answer
        return predict_example(example, residual_tree)

    
# 3.2 All examples of the test data
def decision_tree_predictions(test_df, tree):
    # we predict all the instances in the test data using a loop
    predictions = test_df.apply(predict_example, args=(tree,), axis=1)
    return predictions

## Using the functions

In [92]:
from pprint import pprint


In [93]:
data = pd.read_csv("new_dataset.csv");
data['label'] = data.Attrition
data = data.drop("Attrition", axis=1)

#data.columns = data.columns.str.replace(' ', '_')

data.head()

,Unnamed: 0,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,...,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,label
0,0,0.547619,1.0,0.715820,Sales,0.000000,0.250000,Life Sciences,0.333333,0.0,...,0.00,0.0,0.000,0.0,0.00,0.15,0.222222,0.000000,0.344828,1
1,1,0.814978,0.5,0.185735,Research & Development,0.275862,0.157895,Life Sciences,0.727273,1.0,...,1.00,0.5,0.025,0.5,0.75,0.25,0.388889,0.066667,0.482759,0
2,2,0.613136,1.0,0.915934,Research & Development,0.068966,0.380435,Other,1.000000,1.0,...,0.50,0.0,0.025,0.5,0.75,0.00,0.000000,0.000000,0.000000,1
3,3,0.545855,0.5,0.928610,Research & Development,0.103448,0.793478,Life Sciences,1.000000,0.0,...,0.75,0.0,0.025,0.5,0.75,0.20,0.388889,0.200000,0.000000,0
4,4,0.444950,1.0,0.394188,Research & Development,0.068966,0.173913,Medical,0.181818,1.0,...,1.00,0.5,0.025,0.5,0.75,0.05,0.111111,0.133333,0.137931,0


In [94]:

train_df, test_df = train_test_split(data, test_size=0.2)

test_df.head()

,Unnamed: 0,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,...,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,label
155,155,0.480625,0.0,0.997999,Research & Development,0.034483,0.173913,Technical Degree,1.000000,1.0,...,0.25,0.5,0.025,0.166667,0.75,0.277778,0.388889,0.000000,0.620690,0
318,318,0.447270,1.0,0.813867,Research & Development,0.172414,0.586957,Life Sciences,0.738095,0.0,...,0.50,0.0,0.100,0.333333,0.50,0.111111,0.176471,0.066667,0.137931,0
1068,1068,0.464020,0.5,0.193274,Research & Development,0.068966,0.380435,Medical,0.738095,1.0,...,0.75,0.0,0.325,0.333333,0.50,0.000000,0.000000,0.000000,0.000000,1
350,350,0.698511,1.0,0.362879,Human Resources,0.068966,0.173913,Technical Degree,0.738095,1.0,...,0.75,0.5,0.100,0.833333,0.75,0.083333,0.117647,0.066667,0.000000,0
866,866,0.665012,0.5,0.790378,Sales,0.068966,0.793478,Medical,0.476190,1.0,...,1.00,0.5,0.250,0.333333,0.75,0.000000,0.000000,0.000000,0.000000,0


In [95]:

tree = decision_tree_algorithm(train_df, max_depth=15)

predictions = decision_tree_predictions(test_df, tree)

#test_df.head()

In [96]:
#test_df['label'] = data[data.index.isin(list(test_df.index))]
accuracy_ = calculate_accuracy(predictions, test_df.label)

print("Accuracy = {}".format(accuracy_))


Accuracy = 0.782312925170068


In [97]:
f_score(predictions, test_df.label)

0.2380952380952381

In [98]:
confusion_matrix(predictions, test_df.label)

array([[ 10,  33],
       [ 31, 220]], dtype=int64)

## Random Forest

In [99]:
# creating the dataset for each decision tree of the forest
def bootstrapping(train_df, n_bootstrap):
    bootstrap_indices = np.random.randint(low=0, high=len(train_df), size=n_bootstrap)
    df_bootstrapped = train_df.iloc[bootstrap_indices]
    
    return df_bootstrapped

# creating the forest
def random_forest_algorithm(train_df, n_trees, n_bootstrap, n_features, dt_max_depth):
    forest = []
    for i in range(n_trees):
        # we generate a sub dataset for each tree
        df_bootstrapped = bootstrapping(train_df, n_bootstrap)
        # we train each tree using the sub dataset
        tree = decision_tree_algorithm(df_bootstrapped, max_depth=dt_max_depth, random_subspace=n_features)
        # we add the tree to the forest
        forest.append(tree)
    
    return forest

def random_forest_predictions(test_df, forest):
    df_predictions = {}
    for i in range(len(forest)):
        # we organize every prediction in a dictionary
        column_name = "tree_{}".format(i)
        predictions = decision_tree_predictions(test_df, tree=forest[i])
        df_predictions[column_name] = predictions
    # we create a dataframe from the dictionary of predictions
    df_predictions = pd.DataFrame(df_predictions)
    # we make the final prediction by taking the mode of the predictions, aka majority vote
    random_forest_predictions = df_predictions.mode(axis=1)[0]
    
    return random_forest_predictions


In [100]:
forest = random_forest_algorithm(train_df, n_trees=10, n_bootstrap=250, n_features=None, dt_max_depth=20)
predictions = random_forest_predictions(test_df, forest)
accuracy_ = calculate_accuracy(predictions, test_df.label)

print("Accuracy = {}".format(accuracy_))


Accuracy = 0.8503401360544217


In [101]:
accuracy(test_df.label, predictions)

0.8503401360544217

In [102]:
sensitivity(test_df.label, predictions)

0.04878048780487805

In [103]:
specificity(test_df.label, predictions)

0.9802371541501976

In [104]:
f_score(test_df.label, predictions)

0.08333333333333334

In [105]:
confusion_matrix(test_df.label, predictions)

array([[  2.,  39.],
       [  5., 248.]])

## Grid search but manual

In [133]:
import itertools

In [136]:
def gridsearch(algorithm, parameters):
        # we create a list of all the possible combinations of parameters
        parameter_values = [v for v in parameters.values()]
        combinations = list(itertools.product(*parameter_values))
        # we create a dataframe to store the results
        columns = ["parameters"] + list(parameters.keys()) + ["accuracy"]
        df_results = pd.DataFrame(columns=columns)
        # we loop over all the combinations of parameters
        for c in combinations:
                # we create a dictionary of parameters
                params = dict(zip(parameters.keys(), c))
                # we train the model with the current combination of parameters
                model = algorithm(train_df,**params)
                # we predict the test set
                predictions = decision_tree_predictions(test_df, model)
                # we calculate the accuracy
                accuracy = calculate_accuracy(predictions, test_df.label)
                # we store the results in the dataframe
                df_results.loc[len(df_results)] = [params] + list(params.values()) + [accuracy]
        # we sort the dataframe by accuracy in descending order
        df_results.sort_values(by=["accuracy"], ascending=False, inplace=True)
        df_results.reset_index(inplace=True, drop=True)
        
        return df_results

In [137]:
gridsearch(decision_tree_algorithm, {'max_depth':range(1, 50)})

AttributeError: 'int' object has no attribute 'keys'